## Introduction
This is my second competition submission, where i put together what i have learned during my data science bootcamp and what i have learned through kaggle Data scientist and its courses.

## Table of Content 
    
   
   1. [The Challenge](#cell1)
   2. [Variables Overview](#cell2)
   2. [Importaing relevant libraries](#cell3)
   3. [importing dataset](#cell4)
   4. [Preprocessing dataset](#cell5)
       - Dealing with missing values
       - Data Type check
       - New numerical Dataframe
       - New categorical Dataframe
       
   5. [Feature Selection](#cell6)
       - Numerical Features
       - Categorical Features
       - Creating Dummy Variables for Categorical features 
        
        
   6. [Input and targets](#cell7)
   
   7. [Train, test , Split](#cell8)
   8. [Model Selection](#cell9)
       - Logistic Regression
       - Random Forest 
       - Ridge Classifier
       

## The challange <a id="cell1"></a>



## Variables Overview<a id="cell2"></a>

The dataset is used for this competition is synthetic but based on a real dataset and generated using a CTGAN. The original dataset deals with predicting the amount of an insurance claim. Although the features are anonymized, they have properties relating to real-world features.

## Importaing relevant libraries<a id="cell3"></a>

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


## Importing dataset<a id="cell4"></a>

In [ ]:
raw_data = pd.read_csv('train.csv')
raw_data.head()

## Pre-proccesing <a id="cell5"></a>

In [ ]:
#Checking for Null values 
raw_data.isnull().sum()


In [ ]:
cleaned_data=raw_data

### Data Type Check

In [ ]:
#i check the data types of the features to better categorize between categorical varibles and numerical variables
cleaned_data.dtypes

### New numerical Dataframe

In [ ]:
# Droping all the catigorical featueres and creating a new dataframe with only the target, id and numerical values
data_num = cleaned_data.drop(['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10','cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18'],axis=1)
data_num


### New categorical Dataframe

In [ ]:
# Droping all the numerical featueres and creating a new dataframe with only the target, id and numerical values
data_cat = cleaned_data.drop(['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10'],axis=1)
data_cat

### Numerical Visualazation

### Categorical Features Visualaziton 
The first thing i notic is that a couple of the featus have high unique values, which make it diffcult for creating dummies, so 
i try and see how i can segement these categorial features

In [ ]:
data_cat.describe(include='all')

## Input and targets<a id="cell7"></a>


In [ ]:
targets = cleaned_data['target']
inputs = cleaned_data.drop(['target'], axis=1)

## Train, test , Split<a id="cell8"></a>

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(inputs,targets,test_size= 0.30,random_state=9)

## Feature Enginering  Count Encoding

In [ ]:
from feature_engine.encoding import CountFrequencyEncoder

In [ ]:
enc = CountFrequencyEncoder()

In [ ]:
enc.fit(x_train)

In [ ]:
x_train_enc = enc.transform(x_train)

In [ ]:
x_train_enc 

## Feature Selection 

In [ ]:
from sklearn.feature_selection import SelectKBest

In [ ]:
bestfeatures = SelectKBest(k=20)
fit = bestfeatures.fit(x_train_enc,y_train)

In [ ]:
dfscore = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x_train_enc.columns)

In [ ]:
feature = pd.concat([dfcolumns,dfscore],axis=1)
feature.columns = ['feature','Score']

In [ ]:
#Plot
x_featued_sel = feature.nlargest(20, 'Score')

In [ ]:
sns.barplot(y=x_featued_sel['feature'], x=x_featued_sel['Score'])

In [ ]:
##Dropping all low performing features
x_cleaned = x_train_enc.drop(['cont0','cont4','cont7','cont9','cont10','cat2','cat3','cat5','cat7','cat8','cat10','cat12','cont2'],axis=1)


In [ ]:
x_cleaned

## Scaling 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_cleaned)

In [ ]:
scaler.transform(x_cleaned)

## Model Selection <a id="cell9"></a>

### Logistic model

In [ ]:
from sklearn.linear_model import LogisticRegression
lgr= LogisticRegression()
lgr.fit(x_cleaned,y_train)

In [ ]:
y_pred_lgr = lgr.predict(x_cleaned)


In [ ]:
sc_reg_lgr=confusion_matrix(y_train, y_pred_lgr)
sc_lgr = np.array(sc_reg_lgr)
accuracy_score_lgr = (sc_lgr[0,0]+sc_lgr[1,1])/sc_lgr.sum()
accuracy_score_lgr

## KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

knc = KNeighborsClassifier(n_neighbors=5)
knc.fit(x_cleaned,y_train)

In [ ]:
y_pred_knn = knc.predict(x_cleaned)

In [ ]:
sc_conf_knc=confusion_matrix(y_train, y_pred_knn)
sc_knc = np.array(sc_conf_knc)
accuracy_score_knc = (sc_knc[0,0]+sc_knc[1,1])/sc_knc.sum()
accuracy_score_knc

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(criterion='gini',max_depth=40,max_features='auto')
rfc.fit(x_cleaned,y_train)

In [ ]:
y_pred_rfc = rfc.predict(x_cleaned)
y_pred_rfc

In [ ]:
sc_confr_rfc=confusion_matrix(y_train, y_pred_rfc)
sc_rfc = np.array(sc_confr_rfc)
accuracy_score_rfc = (sc_rfc[0,0]+sc_rfc[1,1])/sc_rfc.sum()
accuracy_score_rfc

## Hyperparameter

In [ ]:
#parameters
from sklearn.model_selection import RandomizedSearchCV
random_grid_par = {
'criterion' : ["gini", "entropy"],
'max_features': ['auto', 'sqrt'],
'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
'min_samples_split': [2, 5, 10],
'min_samples_leaf': [1, 2, 4],
'bootstrap': [True, False]}

In [ ]:
rfc_rg = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid_par, cv = 10, verbose=2, n_jobs = 6)

In [ ]:
rfc_rg.fit(x_cleaned,y_train)

In [ ]:
rfc_rg.best_params_

In [ ]:
rfc_rg.best_score_

### Ridge Classifer

In [ ]:
from sklearn.linear_model import RidgeClassifier
rc = RidgeClassifier()
rc.fit(x_cleaned,y_train)



In [ ]:
y_pred_rc = rc.predict(x_cleaned)
y_pred_rc

In [ ]:
sc_conf_rc=confusion_matrix(y_train, y_pred_rc)
sc_rc = np.array(sc_conf_rc)
accuracy_score_rc = (sc_rc[0,0]+sc_rc[1,1])/sc_rc.sum()
accuracy_score_rc

## SVC

from sklearn.svm import SVC

svc = SVC()
svc.fit(x_cleaned,y_train)



y_pred_svc = rc.predict(x_train)
y_pred_svc

sc_conf_svc=confusion_matrix(y_train, y_pred_svc)
sc_svc = np.array(sc_conf_svc)
accuracy_score_svc = (sc_svc[0,0]+sc_svc[1,1])/sc_svc.sum()
accuracy_score_svc

### Hyper Parameter Tuning SVC

grid = {
    'c': [1. 10, 100, 100]
    'kernel': ['rbf','linear']
    'gama':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
}


grid_search_cv = GridSearchCV(SVC(), param_grid=grid, scoring='accuracy')

## Test data 

In [ ]:
test_data = pd.read_csv('test.csv')

In [ ]:
enc.fit(test_data)
x_test = enc.transform(test_data)

In [ ]:
x_test = x_test.drop(['cont0','cont4','cont7','cont9','cont10','cat2','cat3','cat5','cat7','cat8','cat10','cat12','cont2'],axis=1)

In [ ]:
pred_values = rc.predict(x_test)

In [ ]:
pred_values

In [ ]:
pid = test_data['id']
pid

In [ ]:
output = pd.DataFrame(pid, columns=['id'])
output['target'] = pred_values
output

In [ ]:
output.dtypes

In [ ]:
output.to_csv(r'C:\Users\Hugo\Desktop\output.csv', index = False)